In [154]:
#import packages
from splinter import Browser
from bs4 import BeautifulSoup
import requests
import pandas as pd
import sqlite3

In [45]:
# url for testing
url = "https://www.akc.org/dog-breeds/golden-retriever/"

In [46]:
# get response from url
response = requests.get(url)

In [47]:
# parse html response
soup = BeautifulSoup(response.text, "html.parser")

In [48]:
# drill into the div element of interest
retreiver = soup.find('div', class_='panel-flex__aside')

In [49]:
# parse through the list in the div element and strip the correct piece
attributes = []
for list_ in retreiver.ul:
    for attr in list_:
        try:
            attributes.append(attr.text.strip())
        except:
            attributes.append(attr.strip())

In [50]:
# get rid of empty spaces
attributes  = [a for a in attributes if a is not '']

In [51]:
# result
print(attributes)

['Temperament:', 'Friendly, Intelligent, Devoted', 'AKC Breed Popularity:', 'Ranks 3 of 193', 'Height:', '23-24 inches (male), 21.5-22.5 inches (female)', 'Weight:', '65-75 pounds (male), 55-65 pounds (female)', 'Life Expectancy:', '10-12 years', 'Group:', 'Sporting Group', 'The AKC has grouped all of the breeds that it registers into seven categories, or groups, roughly based on function and heritage. Breeds are grouped together because they share traits of form and function or a common heritage.']


In [52]:
# split up result into keys and values and then turn into dictionary
# all odd values are keys and even values are values
keys = [a for a in attributes if attributes.index(a) % 2 == 0]
values = [a for a in attributes if attributes.index(a) % 2 != 0]
dictionary = dict(zip(keys, values))

In [53]:
print(dictionary)

{'Temperament:': 'Friendly, Intelligent, Devoted', 'AKC Breed Popularity:': 'Ranks 3 of 193', 'Height:': '23-24 inches (male), 21.5-22.5 inches (female)', 'Weight:': '65-75 pounds (male), 55-65 pounds (female)', 'Life Expectancy:': '10-12 years', 'Group:': 'Sporting Group'}


In [54]:
# use dictionary to test data cleaning
# data = [dictionary]

# use file extracted by Ryan
data = pd.read_csv('../resources/dirtydatascraped.csv')
# convert dictionary to dataframe
dog_df = pd.DataFrame(data)

In [55]:
# change index
dog_df.set_index('Breed')

,AKC Breed Popularity,Energy level,Grooming frequency,Group,Height,Image URL,Life Expectancy,Shedding,Temperament,Trainability,Weight
Breed,,,,,,,,,,,
Affenpinscher,Ranks 148 of 193,Regular Exercise,2-3 Times a Week Brushing,Toy Group,9-11.5 inches,https://s3.amazonaws.com/cdn-origin-etr.akc.or...,12-15 years,Seasonal,"Confident, Famously Funny, Fearless",Easy Training,7-10 pounds
Afghan Hound,Ranks 113 of 193,Energetic,Daily Brushing,Hound Group,25-27 inches,https://s3.amazonaws.com/cdn-origin-etr.akc.or...,12-18 years,Infrequent,"Independent, Sweet, Regal",May be Stubborn,50-60 pounds
Airedale Terrier,Ranks 60 of 193,Regular Exercise,2-3 Times a Week Brushing,Terrier Group,23 inches,https://s3.amazonaws.com/cdn-origin-etr.akc.or...,11-14 years,Occasional,"Friendly, Clever, Courageous",Eager to Please,50-70 pounds
Akita,Ranks 47 of 193,Energetic,Daily Brushing,Working Group,"26-28 inches (male), 24-26 inches (female)",https://s3.amazonaws.com/cdn-origin-etr.akc.or...,10-13 years,Seasonal,"Courageous, Dignified, Profoundly Loyal",Eager to Please,"100-130 pounds (male), 70-100 pounds (female)"
Alaskan Malamute,Ranks 58 of 193,Energetic,2-3 Times a Week Brushing,Working Group,"25 inches (male), 23 inches (female)",https://s3.amazonaws.com/cdn-origin-etr.akc.or...,10-14 years,2-3 Times a Week Brushing,"Affectionate, Loyal, Playful",Independent,"85 pounds (male), 75 pounds (female)"
American English Coonhound,Ranks 175 of 193,Energetic,Occasional Bath/Brush,Hound Group,"24-26 inches (male), 23-25 inches (female)",https://s3.amazonaws.com/cdn-origin-etr.akc.or...,11-12 years,Occasional,"Sweet, Mellow, Sociable",Agreeable,45-65 pounds
American Eskimo Dog,Ranks 122 of 193,Energetic,Weekly Brushing,Non-Sporting Group,"9-12 inches (toy), 12-15 inches (miniature), 1...",https://s3.amazonaws.com/cdn-origin-etr.akc.or...,13-15 years,Seasonal,"Playful, Perky, Smart",Eager to Please,"6-10 pounds (toy), 10-20 pounds (miniature), 2..."
American Foxhound,Ranks 186 of 193,Energetic,Occasional Bath/Brush,Hound Group,"22-25 inches (male), 21-24 inches (female)",https://s3.amazonaws.com/cdn-origin-etr.akc.or...,11-13 years,Seasonal,"Independent, Easy-Going, Sweet-Tempered",Independent,"65-70 pounds (male), 60-65 pounds (female)"
American Staffordshire Terrier,Ranks 85 of 193,Regular Exercise,Occasional Bath/Brush,Terrier Group,"18-19 inches (male), 17-18 inches (female)",https://s3.amazonaws.com/cdn-origin-etr.akc.or...,12-16 years,Occasional,"Confident, Smart, Good-Natured",Agreeable,"55-70 pounds (male), 40-55 pounds (female)"


In [56]:
# turn temprament values into referencable array
dog_df['temperament'] = dog_df['Temperament'].str.split(',')

In [57]:
dog_df['Life Expectancy']

0         12-15 years
1         12-18 years
2         11-14 years
3         10-13 years
4         10-14 years
5         11-12 years
6         13-15 years
7         11-13 years
8         12-16 years
9         10-14 years
10        11-13 years
11        12-15 years
12        12-16 years
13        10-13 years
14        12-15 years
15        11-15 years
16        12-15 years
17        12-14 years
18        13-14 years
19        10-15 years
20        12-14 years
21        11-16 years
22        14-16 years
23        12-14 years
24        12-14 years
25        12-13 years
26         7-10 years
27        14-15 years
28        10-12 years
29        10-12 years
            ...      
157        8-11 years
158          12 years
159       12-14 years
160       12-14 years
161       13-16 years
162       10-18 years
163       12-14 years
164       13-15 years
165       12-14 years
166    12 to 14 years
167       12-14 years
168       13-15 years
169       12-14 years
170       13-16 years
171       

In [58]:
# clean data 
dog_df[dog_df['Life Expectancy'].isnull()]
# fill in missing value for Rhodesian Ridgeback
dog_df.at[152, 'Life Expectancy'] = '10-12 years'


In [59]:
# find bad data
dog_df[dog_df['Life Expectancy'].str.contains('to')]
# fix bad data
dog_df.at[166, 'Life Expectancy'] = '12-14 years'

In [60]:
# find bad data
dog_df[dog_df['Life Expectancy'].str.contains('Late')]
# fix bad data
dog_df.at[149, 'Life Expectancy'] = '15-17 years'

In [61]:

dog_df['Life_Expectancy'] = pd.DataFrame(dog_df['Life Expectancy'].str.split(' '))
avg_life_expectancy = []
for index, row in dog_df.iterrows():
    lifespan = row['Life_Expectancy'][0].split('-')

    try:
        avg_life_expectancy.append((int(lifespan[0])+int(lifespan[1])/2))
    except: 
        avg_life_expectancy.append(float(lifespan[0]))
        
#     print(lifespan)
dog_df['avg_life_expectancy'] = avg_life_expectancy
# dog_df['range'] = pd.DataFrame(dog_df['Life_Expectancy'].values.tolist(), index=dog_df.index)
# dog_df.Life_Expectancy.values.tolist()
# 

In [62]:
len(avg_life_expectancy)

187

In [63]:
# Debug this for some reason the split(' ') is not working correctly
# might be because the try-except is failing
# cleazn up null values
dog_df[dog_df['Weight'].isnull()]

,Breed,AKC Breed Popularity,Energy level,Grooming frequency,Group,Height,Image URL,Life Expectancy,Shedding,Temperament,Trainability,Weight,temperament,Life_Expectancy,avg_life_expectancy
13,Australian Kelpie,NaN,Energetic,Weekly Brushing,Foundation Stock Service,17-20 inches,https://s3.amazonaws.com/cdn-origin-etr.akc.or...,10-13 years,Regularly,"Loyal, Alert, Intelligent",Eager to Please,NaN,"[Loyal, Alert, Intelligent]","[10-13, years]",16.5
176,Toy Fox Terrier,Ranks 111 of 193,Energetic,Weekly Brushing,Toy Group,8.5-11.5 inches,https://s3.amazonaws.com/cdn-origin-etr.akc.or...,13-15 years,Occasional,"Friendly, Alert, Intelligent",Eager to Please,NaN,"[Friendly, Alert, Intelligent]","[13-15, years]",20.5


In [64]:
# insert values into null rows
dog_df.at[13, 'Weight'] = "31-46 pounds (male), 31-46 pounds (female)"
dog_df.at[176, 'Weight'] = "3.5-7 pounds"

In [65]:
dog_df[dog_df['Weight'].str.contains('under')]

,Breed,AKC Breed Popularity,Energy level,Grooming frequency,Group,Height,Image URL,Life Expectancy,Shedding,Temperament,Trainability,Weight,temperament,Life_Expectancy,avg_life_expectancy
19,Beagle,Ranks 6 of 193,Energetic,2-3 Times a Week Brushing,Hound Group,"13 inches & under, 13-15 inches",https://s3.amazonaws.com/cdn-origin-etr.akc.or...,10-15 years,Seasonal,"Friendly, Curious, Merry",Agreeable,"under 20 pounds (13 inches & under), 20-30 pou...","[Friendly, Curious, Merry]","[10-15, years]",17.5
67,Dachshund,Ranks 12 of 193,Regular Exercise,Weekly Brushing,Hound Group,"8-9 inches (standard), 5-6 inches (miniature)",https://s3.amazonaws.com/cdn-origin-etr.akc.or...,12-16 years,Occasional,"Friendly, Curious, Spunky",Agreeable,"16-32 pounds (standard), 11 pounds & under (mi...","[Friendly, Curious, Spunky]","[12-16, years]",20.0
84,French Bulldog,Ranks 4 of 193,Calm,Occasional Bath/Brush,Non-Sporting Group,11-13 inches,https://s3.amazonaws.com/cdn-origin-etr.akc.or...,10-12 years,Regularly,"Adaptable, Playful, Smart",Agreeable,under 28 pounds,"[Adaptable, Playful, Smart]","[10-12, years]",16.0
120,Maltese,Ranks 37 of 193,Regular Exercise,Daily Brushing,Toy Group,7-9 inches,https://s3.amazonaws.com/cdn-origin-etr.akc.or...,12-15 years,Infrequent,"Gentle, Playful, Charming",Agreeable,under 7 pounds,"[Gentle, Playful, Charming]","[12-15, years]",19.5


In [66]:
dog_df.at[84, 'Weight'] = "28 pounds and under"
dog_df.at[120, 'Weight'] = "7 pounds and under"
dog_df.at[19, 'Weight'] = "22-24 pounds (male), 20-22 pounds (female)"
dog_df.at[97, 'Weight'] = "16-33 pounds"

In [67]:
dog_df[dog_df['Weight'].str.contains('Proportion')]

,Breed,AKC Breed Popularity,Energy level,Grooming frequency,Group,Height,Image URL,Life Expectancy,Shedding,Temperament,Trainability,Weight,temperament,Life_Expectancy,avg_life_expectancy
50,Cane Corso,Ranks 32 of 193,Energetic,Occasional Bath/Brush,Working Group,"25-27.5 inches (male), 23.5-26 inches (female)",https://s3.amazonaws.com/cdn-origin-etr.akc.or...,9-12 years,Occasional,"Affectionate, Intelligent, Majestic",Agreeable,Proportionate to height,"[Affectionate, Intelligent, Majestic]","[9-12, years]",15.0


In [68]:
dog_df.at[50, 'Weight'] = "99-110 pounds (male), 88-99 pounds (female)"

In [69]:
dog_df[dog_df['Weight'].str.contains('exceeding')]

,Breed,AKC Breed Popularity,Energy level,Grooming frequency,Group,Height,Image URL,Life Expectancy,Shedding,Temperament,Trainability,Weight,temperament,Life_Expectancy,avg_life_expectancy
57,Chihuahua,Ranks 33 of 193,Regular Exercise,Occasional Bath/Brush,Toy Group,5-8 inches,https://s3.amazonaws.com/cdn-origin-etr.akc.or...,14-16 years,Occasional,"Charming, Graceful, Sassy",Independent,not exceeding 6 pounds,"[Charming, Graceful, Sassy]","[14-16, years]",22.0


In [70]:
dog_df.at[57,'Weight'] = '3.3-6.6 pounds'

In [71]:
dog_df[dog_df['Weight'].str.contains('around')]

,Breed,AKC Breed Popularity,Energy level,Grooming frequency,Group,Height,Image URL,Life Expectancy,Shedding,Temperament,Trainability,Weight,temperament,Life_Expectancy,avg_life_expectancy
164,Silky Terrier,Ranks 112 of 193,Regular Exercise,2-3 Times a Week Brushing,Toy Group,9-10 inches,https://s3.amazonaws.com/cdn-origin-etr.akc.or...,13-15 years,Occasional,"Friendly, Quick, Keenly Alert",Agreeable,around 10 pounds,"[Friendly, Quick, Keenly Alert]","[13-15, years]",20.5


In [72]:
dog_df.at[164, 'Weight'] = '10 pounds'

In [73]:
dog_df[dog_df['Weight'].str.contains('up to')]

,Breed,AKC Breed Popularity,Energy level,Grooming frequency,Group,Height,Image URL,Life Expectancy,Shedding,Temperament,Trainability,Weight,temperament,Life_Expectancy,avg_life_expectancy
137,Pekingese,Ranks 92 of 193,Calm,2-3 Times a Week Brushing,Toy Group,6-9 inches,https://s3.amazonaws.com/cdn-origin-etr.akc.or...,12-14 years,Seasonal,"Affectionate, Loyal, Regal in Manner",May be Stubborn,up to 14 pounds,"[Affectionate, Loyal, Regal in Manner]","[12-14, years]",19.0
138,Pembroke Welsh Corgi,Ranks 13 of 193,Energetic,Weekly Brushing,Herding Group,10-12 inches,https://s3.amazonaws.com/cdn-origin-etr.akc.or...,12-13 years,Regularly,"Affectionate, Smart, Alert",Agreeable,"up to 30 pounds (male), up to 28 pounds (female)","[Affectionate, Smart, Alert]","[12-13, years]",18.5


In [74]:
dog_df.at[137, 'Weight'] = "14 pounds or less"
dog_df.at[138, 'Weight'] = "30 pounds (male), 28 pounds (female)"

In [75]:
# split the male and female values into an array
dog_df['Weight2'] = pd.DataFrame(dog_df['Weight'].str.split(','))
avg_wt = []

# for every row, go through and strip out the weight values based on formatting
for index, row in dog_df.iterrows():
    
#     if the last step split things correctly, the first element should be male data
    try: 
        male_weight = row['Weight2'][0]
        male_weight_split = male_weight.split(' ')[0].split('-')
#         print(male_weight_split)
        
#         Sometimes there is only one number for the male and not a range
        try:
            male_avg_weight = (float(male_weight_split[0]) + float(male_weight_split[1]))/2
        except:
            male_avg_weight = float(male_weight_split)
#         print(male_avg_weight) 
#     replace all bad data with zero    
    except:
        male_weight = row['Weight2']
        male_avg_weight = 0
#         print(None)
    
#     if the first step outside loop was successful, the second element should be female data
    try:
        female_weight = row['Weight2'][1]
        female_weight_split = female_weight.split(' ')[0].split('-')
        
#         sometimes the value is just a number and not a range
        try:
            female_avg_weight = (float(female_weight[0]) + float(female_weight[1]))/2
        except:
            female_avg_weight = (float(female_weight))
            
#         if we fail to average the female and male average weights then we just keep the male average
        try:
            avg_weight = (male_avg_weight + female_avg_weight)/2
            avg_wt.append(avg_weight)
        except:
            avg_weight = male_avg_weight
            avg_wt.append(avg_weight)
    
    except:
            avg_weight = male_avg_weight
            avg_wt.append(avg_weight)
            
#     print(avg_wt)
dog_df['avg_weight'] = avg_wt

In [76]:
# dog_df['weight_male_low'] = int(dog_df['weight_male'].str.split(' ')[0][0].split('-')[0])
# dog_df['weight_male_high'] = int(dog_df['weight_male'].str.split(' ')[0][0].split('-')[1])

In [77]:
# dog_df['weight_female_low'] = int(dog_df['weight_female'].str.split(' ')[0][1].split('-')[0])
# dog_df['weight_female_high'] = int(dog_df['weight_female'].str.split(' ')[0][1].split('-')[1])

In [78]:
# dog_df['avg_weight_male_lbs'] = dog_df[['weight_male_low', 'weight_male_high']].mean(axis=1)
# dog_df['avg_weight_female_lbs'] = dog_df[['weight_female_low', 'weight_female_high']].mean(axis=1)

In [79]:
dog_df[dog_df['Height'].str.contains('Dogs')]

,Breed,AKC Breed Popularity,Energy level,Grooming frequency,Group,Height,Image URL,Life Expectancy,Shedding,Temperament,Trainability,Weight,temperament,Life_Expectancy,avg_life_expectancy,Weight2,avg_weight
112,Kooikerhondje,Ranks 150 of 193,Regular Exercise,Weekly Brushing,Sporting Group,"Dogs 16"", Females 15""",https://s3.amazonaws.com/cdn-origin-etr.akc.or...,12-15 years,Seasonal,"Friendly, Alert, Quick",Eager to Please,20 - 30 lbs,"[Friendly, Alert, Quick]","[12-15, years]",19.5,[20 - 30 lbs],0.0


In [80]:
dog_df.at[112,'Height'] = "16 inches (male), 15 inches (female)"

In [81]:
dog_df[dog_df['Height'].str.contains('minimum')]

,Breed,AKC Breed Popularity,Energy level,Grooming frequency,Group,Height,Image URL,Life Expectancy,Shedding,Temperament,Trainability,Weight,temperament,Life_Expectancy,avg_life_expectancy,Weight2,avg_weight
88,German Wirehaired Pointer,Ranks 63 of 193,Needs Lots of Activity,Weekly Brushing,Sporting Group,"24-26 inches (male), minimum 22 inches (female)",https://s3.amazonaws.com/cdn-origin-etr.akc.or...,14-16 years,Regularly,"Affectionate, Eager, Enthusiastic",Eager to Please,50-70 pounds,"[Affectionate, Eager, Enthusiastic]","[14-16, years]",22.0,[50-70 pounds],60.0
105,Irish Wolfhound,Ranks 76 of 193,Regular Exercise,Weekly Brushing,Hound Group,"32 inches minimum (male), 30 inches minimum (f...",https://s3.amazonaws.com/cdn-origin-etr.akc.or...,6-8 years,Seasonal,"Courageous, Dignified, Calm",Independent,"120 pounds (male), 105 pounds (female)","[Courageous, Dignified, Calm]","[6-8, years]",10.0,"[120 pounds (male), 105 pounds (female)]",0.0
111,Komondor,Ranks 173 of 193,Regular Exercise,Specialty/Professional,Working Group,"27.5 minimum inches (male), 25.5 minimum inche...",https://s3.amazonaws.com/cdn-origin-etr.akc.or...,10-12 years,Specialty/Professional,"Loyal, Dignified, Brave",Independent,"100 pounds or more (male), 80 pounds or more (...","[Loyal, Dignified, Brave]","[10-12, years]",16.0,"[100 pounds or more (male), 80 pounds or more...",0.0
173,Tibetan Mastiff,Ranks 131 of 193,Regular Exercise,2-3 Times a Week Brushing,Working Group,"minimum 26 inches (male), minimum 24 inches (f...",https://s3.amazonaws.com/cdn-origin-etr.akc.or...,10-12 years,Seasonal,"Independent, Reserved, Intelligent",Independent,"90-150 pounds (male), 70-120 pounds (female)","[Independent, Reserved, Intelligent]","[10-12, years]",16.0,"[90-150 pounds (male), 70-120 pounds (female)]",120.0


In [82]:
dog_df.at[173, 'Height'] = "26 minimum inches (male), 24 minimum inches (female)"

In [83]:
# initial split of Height column
dog_df['Height2'] = pd.DataFrame(dog_df['Height'].str.split(','))
avg_ht = []

# loop through all the rows to split up the male height values
for index, row in dog_df.iterrows():
    male_height = row['Height2'][0]
    male_height_split = male_height.split(' ')[0].split('-')
#     print(male_height_split)
#     try to get an average value and if you can't average, just take first value
    try:
        avg_male_height = (float(male_height_split[0]) + float(male_height_split[1]))/2
    except:
        avg_male_height = float(male_height_split[0])

#     print(avg_male_height)
#     try to get a value from the female half and if that fails there wasn't female data
    try:
        female_height = row['Height2'][1]
        female_height_split = female_height.split(' ')[0].split('-')
#     print(male_height_split)
        try:
            avg_female_height = (float(female_height_split[0]) + float(female_height_split[1]))/2
        except:
            avg_female_height = float(female_height_split[0])
        
        try:
            avg_height = (avg_female_height + avg_male_height)/2
            avg_ht.append(avg_height)
        except:
            avg_height = avg_male_height
            avg_ht.append(avg_height)
    
    except:
        avg_height = avg_male_height
        avg_ht.append(avg_height)
# print(avg_ht)
dog_df['avg_height'] = avg_ht


In [147]:
# select new dataframe with cleaned data and change column names
doggy_df = dog_df[['Breed','Group', 'temperament', 'avg_life_expectancy', 'avg_height', 'avg_weight', 'Energy level', 'Grooming frequency']]
doggy_df = doggy_df.rename(columns={'Breed': 'breed','Group':'group', 'avg_life_expectancy': 'life_expectancy_years', 'avg_height': 'avg_height_inch', 'avg_weight': 'avg_weight_pounds', 'Energy level': 'energy', 'Grooming frequency': 'grooming'})

In [148]:
# change index to the breeds
doggy_df.set_index('breed')

,group,temperament,life_expectancy_years,avg_height_inch,avg_weight_pounds,energy,grooming
breed,,,,,,,
Affenpinscher,Toy Group,"[Confident, Famously Funny, Fearless]",19.5,10.250,8.50,Regular Exercise,2-3 Times a Week Brushing
Afghan Hound,Hound Group,"[Independent, Sweet, Regal]",21.0,26.000,55.00,Energetic,Daily Brushing
Airedale Terrier,Terrier Group,"[Friendly, Clever, Courageous]",18.0,23.000,60.00,Regular Exercise,2-3 Times a Week Brushing
Akita,Working Group,"[Courageous, Dignified, Profoundly Loyal]",16.5,27.000,115.00,Energetic,Daily Brushing
Alaskan Malamute,Working Group,"[Affectionate, Loyal, Playful]",17.0,25.000,0.00,Energetic,2-3 Times a Week Brushing
American English Coonhound,Hound Group,"[Sweet, Mellow, Sociable]",17.0,25.000,55.00,Energetic,Occasional Bath/Brush
American Eskimo Dog,Non-Sporting Group,"[Playful, Perky, Smart]",20.5,10.500,8.00,Energetic,Weekly Brushing
American Foxhound,Hound Group,"[Independent, Easy-Going, Sweet-Tempered]",17.5,23.500,67.50,Energetic,Occasional Bath/Brush
American Staffordshire Terrier,Terrier Group,"[Confident, Smart, Good-Natured]",20.0,18.500,62.50,Regular Exercise,Occasional Bath/Brush


In [167]:
doggy_df.to_csv('../resources/clean_dogs.csv')